# INFO 212: Data Science Programming 1
___

## Week 7: Data Cleaning and Preparation
---

**Agenda**
- Missing values are None, numpy.nan
- DataFrame.isnull() detects missing values
- DataFrame.notnull() detects non-missing values
- DataFrame.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)[source]
- DataFrame.fillna(value=None, method=None, axis=None, inplace=False, limit=None, downcast=None)
- DataFrame.transform() performs operations element-wise
- duplicated() returns a boolean Series indicating whether each row is a duplicate
- drop_duplicates() returns a DataFrame where the duplicated array is removed:
- Transform data by applying function
- data.replace([-999, -1000], np.nan): replace values
- data.index.map(transform): rename index
- data.rename(index=str.title, columns=str.upper): rename index and columns

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from google.colab import data_table
data_table.enable_dataframe_formatter

<function google.colab.data_table.enable_dataframe_formatter()>

# Handling Missing Data

Missing data occurs commonly in many data analysis applications.
All of the descriptive statistics on pandas objects exclude missing data by default. For numeric data, pandas uses the floating-point value NaN (Not a Number) to represent missing data. We call this a sentinel value that can be easily detected.

## `isnull()`: detects missing values
```
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])

string_data.isnull()

string_data.isnull().sum()
```

In [4]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])

string_data.isnull()

string_data.isnull().sum()

1

## Exercise:
Run and test the above code.

In [5]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])

string_data.isnull()

string_data.isnull().sum()

1

## Reasons for Missing Values

Before we start treating the missing values, it is important to understand the various reasons for the missingness in data. Broadly speaking, there can be three possible reasons: (references: Wikipedia: https://en.wikipedia.org/wiki/Missing_data ; Flexible Imputation of Missing Data by R: https://stefvanbuuren.name/fimd/sec-MCAR.html)

1. Missing Completely at Random (MCAR): The missing values on a given variable (Y) are not associated with other variables in a given data set or with the variable (Y) itself. In other words, there is no particular reason for the missing values. An example of MCAR is a weighing scale that ran out of batteries. Some of the data will be missing simply because of bad luck.

2. Missing at Random (MAR): MAR occurs when the missingness is not random, but where missingness can be fully accounted for by variables where there is complete information. For example, when placed on a soft surface, a weighing scale may produce more missing values than when placed on a hard surface. Such data are thus not MCAR. If, however, we know surface type and if we can assume MCAR within the type of surface, then the data are MAR.

3. Missing Not at Random (MNAR): Missingness depends on unobserved data or the value of the missing data itself. MNAR means that the probability of being missing varies for reasons that are unknown to us. For example, the weighing scale mechanism may wear out over time, producing more missing data as time progresses, but we may fail to note this.

## Dealing with Missing Data
Missing data are often denoted by a "na, n/a, N/a, N/A (not applicable)" or "Nan, NaN, NAN (not a number)". The denomination depends on the used progamming langauge (NaN is the most common in python language for machine learning). In general, there are two approaches to handle missing data: Deletion and Imputation.

## Deletions

![](https://imgur.com/tBvdfyX.png)

Deletion means to delete the missing values from a dataset. Deletions are further categorized into three types:

### Pairwise Deletion

>Parwise Deletion is used when values are missing completely at random i.e MCAR. During Pairwise deletion, only the missing values are deleted. All operations in pandas like mean,sum etc intrinsically skip missing values.

### Listwise Deletion/ Dropping rows

>During Listwise deletion, complete rows (which contain the missing values) are deleted. As a result, it is also called Complete Case deletion. Like Pairwise deletion, listwise deletions are also only used for MCAR values.

### Dropping complete columns

>If a column contains a lot of missing values, say more than 80%, and the feature is not significant, you might want to delete that feature. However, again, it is not a good methodology to delete data.

  
## `dropna()`: Drops missing values; for Series, it returns non-null data.

```
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])

data.dropna()
```

## Exercise:
Run and test the above code.

In [ ]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])

data.dropna()

## `notnull()`: returns list of boolean values indicating whether an item is null or not

```
data.notnull()
```

##  `dropna()` With DataFrame objects, things are a bit more complex.  You may want to drop rows or columns that are all NA or only those containing any NAs. `dropna` by default drops any row containing a missing value.

```
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])

data.dropna()

data.dropna(axis=1)

data.isnull().sum()

data.isnull().sum().sum()

```

## Exercise:
Run and test the above code.

## Passing how='all' will only drop rows that are all NA:

```
data.dropna(how='all')

data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])

data[3] = NA

data.dropna(axis=1, how='all')
```

## Exercise:
Run and test the above code.

## Suppose you want to keep only rows containing a certain number of observations. You can indicate this with the thresh argument as `dropna(thresh=2)`.


```
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA

df.dropna()
df.dropna(thresh=2)
```

## Exercise:
Run and test the above code.

## Imputations

![](https://imgur.com/bL0iHde.png)

>Imputation refers to replacing missing data with substituted values. There are a lot of ways in which the missing values can be imputed depending upon the nature of the problem and data. Dependng upon the nature of the problem, imputation techniques can be broadly they can be classified as follows:


### Basic Imputation Techniques
  
  - Imputating with a constant value
  - Imputation using the statistics (mean, median or most frequent) of each column in which the missing values are located

For this we shall use the `The SimpleImputer` class from sklearn.

### Filling In Missing Data
For most purposes, the `fillna` method is the workhorse function to use. Calling `fillna` with a constant replaces missing values with that value:

```
data = pd.DataFrame([[1., 2, 3.], [4., NA, NA],
                     [NA, NA, NA], [NA, 8, 9]])


data.fillna(0)

```

Calling fillna with a dict, you can use a different fill value for each column:

```
data.fillna({0: 5, 1: 0.5, 2: 0})

data.fillna(data.mean(), inplace = True)

```


## Exercise:
Run and test the above code.

In [8]:
data = pd.DataFrame([[1., 2, 3.], [4., NA, NA],
                     [NA, NA, NA], [NA, 8, 9]])


data.fillna(0)


NameError: name 'NA' is not defined

### fillna returns a new object, but you can modify the existing object in-place:

```
_ = df.fillna(0, inplace=True)

df.fillna({1:0.3, 2:0.5}, inplace=True)

```

## Exercise:
Run and test the above code.

### With fillna you can do lots of other things with a little creativity. For example, you might pass the mean or median value of a Series:

```
data = pd.Series([1., NA, 3.5, NA, 7])

data.fillna(data.median())

```

## We can also fill in the missing value using the previous or next value by `ffill` or `bfill`.
```
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA

data.fillna(method='bfill')

df.fillna(method='ffill', limit=2)
```

# Data Transformation

## The transform() function:
- The transform() function in pandas is used to perform operations on each element of a series or column in a DataFrame.
- Unlike apply(), transform() returns an object that is the same size as the input.
- This makes it particularly useful for filling missing values.
- The transform() function is often used with groupby objects to perform operations within groups while maintaining the original DataFrame shape.
```
df = pd.DataFrame({'A': ['foo', 'bar', 'foo', 'bar', 'foo', 'bar', ],
                 'B': [1, np.nan, 2, 4, np.nan, 2]})
B_imputed = df.groupby('A')['B'].transform(lambda x: x.fillna(x.mean()))
```

## Exercise:
Run and test the above code.

In [10]:
df = pd.DataFrame({'A': ['foo', 'bar', 'foo', 'bar', 'foo', 'bar', ],
               'B': [1, np.nan, 2, 4, np.nan, 2]})
df

,A,B
0,foo,1.0
1,bar,NaN
2,foo,2.0
3,bar,4.0
4,foo,NaN
5,bar,2.0


In [ ]:
B_imputed = df.groupby('A')['B'].transform(lambda x: x.fillna(x.mean())) #transforms the dataset into 2 classes and fills in the NaN with its class' mean

## Removing Duplicates


```
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})

```

## The DataFrame method `duplicated()` returns a boolean Series indicating whether each row is a duplicate (has been observed in a previous row) or not:
```
data.duplicated()

```

In [11]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data


,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [12]:
data.duplicated()

,0
0,False
1,False
2,False
3,False
4,False
5,False
6,True


### drop_duplicates returns a DataFrame where the duplicated array is removed:
```
data.drop_duplicates()
```

In [13]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


### drop_duplicates can take a column as a parameter.

```
data['v1'] = range(7)

data.drop_duplicates(['k1', 'k2'])

```

In [14]:
data['v1'] = range(7)

data.drop_duplicates(['k1', 'k2'])

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5


### duplicated and drop_duplicates by default keep the first observed value combination. Passing keep='last' will return the last one:

```
data.drop_duplicates(['k1', 'k2'], keep='last')

data.drop_duplicates(keep='last')

```



In [15]:
data.drop_duplicates(['k1', 'k2'], keep='last')

data.drop_duplicates(keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


## Transforming Data Using a Function or Mapping
### For many datasets, you may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame. Consider the following data collected about various kinds of meat:

```
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

```

### Suppose we wanted to add a column indicating the type of animal that each food came from.
### Step 1: Write down a mapping of each distinct meat type to the kind of animal:

```
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}
```

### Step 2: Normalize the text if the map only uses normalized lower-case strings.

```
data['food'].str.lower()

lowercased = data['food'].str.lower()
```

### Step 3: Apply the map method on a Series by accepting a function or dict-like object containing a mapping

```
data['animal'] = data['food'].str.lower().map(meat_to_animal)
```

## We could also have passed a function that does all the work:
```
data['food'].map(lambda x: meat_to_animal[x.lower()])
```

# Replacing Values
Filling in missing data with the fillna method is a special case of more general value
replacement. As you’ve already seen, map can be used to modify a subset of values in
an object.
## `replace` provides a simpler and more flexible way to do so. Let’s consider this Series:

```
data = pd.Series([1., -999., 2., -999., -1000., 3.])
```

The -999 values might be sentinel values for missing data. To replace these with NA
values that pandas understands, we can use replace, producing a new Series (unless
you pass inplace=True):

```
data1 = data.replace([-999, -1000], np.nan)

data1.fillna(data1.mean())
```

## Exercise:
Run and test the above code.

If you want to replace multiple values at once, you instead pass a list and then the
substitute value:

```
data.replace([-999, -1000], np.nan)
```

To use a different replacement for each value, pass a list of substitutes:

```
data.replace([-999, -1000], [np.nan, 0])
```

The argument passed can also be a dict:

```
data.replace({-999: np.nan, -1000: 0})
```

## Renaming Axis Indexes
### Like values in a Series, axis labels can be similarly transformed by a function or mapping of some form to produce new, differently labeled objects.
### We can also modify the axes in-place without creating a new data structure. Here’s a simple example:

```
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
```

### We can modify the index of the DataFrame in-place:

```
transform = lambda x: x[:4].upper()
data.index = data.index.map(transform)
```

### If we want to create a transformed version of a dataset without modifying the original, a useful method is rename:

```
data.rename(index=str.title, columns=str.upper)

data.rename(columns={"one":'column1'})
```

### Notably, rename can be used in conjunction with a dict-like object providing new values for a subset of the axis labels:

```
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})
```

### rename saves you from the chore of copying the DataFrame manually and assigning to its index and columns attributes. Should you wish to modify a dataset in-place, pass inplace=True:

```
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
```